Tutorial_link:

https://medium.com/data-and-beyond/complete-guide-to-building-bert-model-from-sratch-3e6562228891

# Implementing BERT From Scratch
 

BERT(Bidirectional Encoder Representations from Transformers) is a language represtation model developed by Google. 
A Language Representation Model refers to a computational model designed to understand, generate, and represent human language in a meaningful way for tasks such as translation, sentiment analysis, or conversation. These model are pretrained on human language data to understand human language firts,then its fine-tuned to do specific task. 

## How BERT WORK:

1. BERT looks a sequence to predict word in both left-to-right and right-to-left. For example a sentence like "A girl play in ___ with football ". Other model would take only consider either the first part or second part to predict playground for blank but BERT consider both the part.

2. Instead of predicting next word,bert randomly mask word in a sentence then takes account full account of sentcence undertand its context to preidict the mask word.

3. BERT use encoder part of a transformer to understand the context of a language. Words are first converted into tokens.Tokens are neumerical represtation of a word. Tokens are uesd as a input for encoder.Tokens are converted into vector for neural networks to process.

4. BERT needs the input to be massaged and decorated with some extra metadata:

**Token Embeddings**: [CLS] token is added to the input word token at the beginning of the first sentence and a [SEP] token is inserted at the end of each sentence.

**Segment Embeddings**: A marker indicating Sentence A or Sentence B is added to each token. This allows the encoder to distinguish between sentences.

**Positional Embeddings**: A positional embedding is added to each token to indicate its position in the sentence.

![](2024-09-16-12-16-13.png)

**Training** : 
After embedding data,model is ready to be trained. Bert is traing using this two strategies:

4. Masked Language Model (MLM): Bert goes through entire sequence tries to predict Masked word based on the context of non-masked word and traiend along the way.

5. Next Sentence Prediction (NSP): Bert takes a pair of sentences and predict if the second sentence comes next of first sentence. 


## Architecture:
BERT-Base: 12-layer, 768-hidden-nodes, 12-attention-heads, 110M parameters

BERT-Large: 24-layer, 1024-hidden-nodes, 16-attention-heads, 340M parameters


In [15]:
with open('./datasets/movie_conversations.tsv','r') as tsv_file:
    with open('./datasets/movie_conversations.txt','w') as txt_file:
        for line in tsv_file:
            txt_file.write(line.replace('\t',' '))

In [16]:
with open('./datasets/movie_lines.tsv','r') as tsv_file:
    with open('./datasets/movie_lines.txt','w') as txt_file:
        for line in tsv_file:
            txt_file.write(line.replace('\t',' '))

In [12]:
import os
import pandas as pd
from pathlib import Path
import torch
import re
import random
import transformers, datasets
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer
import tqdm
from torch.utils.data import Dataset, DataLoader
import itertools
import math
import torch.nn.functional as F
import numpy as np
from torch.optim import Adam

In [22]:
MAX_LEN=64

corpus_movie_conv="./datasets/movie_conversations.txt"
corpus_movie_lines ="./datasets/movie_lines.txt"

with open(corpus_movie_conv,'r',encoding='iso-8859-1') as c:
    conv=c.readlines()
with open(corpus_movie_lines,'r',encoding='iso-8859-1') as l:
    lines=l.readlines()

lines_dic ={}

for line in lines:
    objects =line.split(" +++$+++ ")
    lines_dic[objects[0]]= objects[-1]

pairs=[]
for con in conv:
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs=[]

        if i== len(ids) -1:
            break

        first=lines_dic[ids[i]].strip()
        second=lines_dic[ids[i+1]].strip()

        qa_pairs.append(' '.join(first.spit()[:MAX_LEN]))
        qa_pairs.append(' '.join(second.split()[:MAX_LEN]))
        pairs.append(qa_pairs)

SyntaxError: invalid syntax (<string>, line 1)

## wordpiece Embeddings:
 Tokenaization is the process of breaking down text into samller units called "tokens",which are then converted into a numerical representation.

 In bert words are represetned through a vector which is stored in lookuptable.For unknown word it split the word into known subword which embeddings are stored in the lookuptable.

In [1]:
#wordPiece Tokenizer
os.mkdir('./data')
text_data=[]
file_count=0

for sample in tqdm.tqdm(x[0] for x in pairs):
    text_data.append(sample)

    if len(text_data) == 10000:
        with open(f'./data/text_{file_count}.txt','w',encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data=[]
        file_count +=1

paths = [str(x) for x in Path('./data').glob('**/*.txt')]

tokenizer= BertWordPieceTokenizer(
    clean_text= True,
    handle_chinese_chars = False,
    strip_accents =False,
    lower_case =True
)

tokenizer.train(
    files=paths,
    vocab_size = 30_000,
    min_frequency=5,
    limit_alphabet =1000,
    wordpicese_prefix='##',
    special_tokens=['[PAD]','[CLS]','[SEP]','[MASK]','[UNK]']
)

os.mkdir('./bert-it-1')
tokenizer.save_model('./bert-it-1','bert-it')
tokenizer = BertTokenizer.from_pretrained('./bert-it-1/bert-it-vocab.txt',local_files_only= True)

NameError: name 'os' is not defined

In [ ]:
class BERTDataset(Dataset):
    def __init__(self,data_pair,tokenizer,seq_len=64) :
        self.tokenizer = tokenizer
        self.seq_len =seq_len
        self.corpus_lines =len(data_pair)

    def __len__(self):
        return self.corpus_lines
    
    def __getitem__(self, item) :
        t1,t2,is_next_lebel= self.get_sent(item)

        t1_random,t1_lebel=self.random_word(t1)
        t2_random,t1_lebel =self.random_word(t2)

        t1=[self.tokenizer.vocab['[CLS]']]+t1_random+[self.tokenizer.vocab['[SEP]']]
        t2=t2_random+[self.tokenizer.vocab['[PAD]']]
        t1_label=[self.tokenizer.vocab['[PAD]']]+t1_label+[self.tokenizer.vocab['[PAD]']]
        t2_label= t2_label+[self.tokenizer.vocab['[PAD]']]

        segment_label=([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])

        bert_input=(t1+t2)[:self.seq_len]
        bert_label=(t1_label+t2_label)[:self.seq_len]
        padding=[self.tokenizer.vocab['[PAD]'] for _ in range(self.seq_len -len(bert_input))]
        bert_input.extend(padding),bert_label.extend(padding),segment_label.extend(padding)

        output ={
            "bert_input": bert_input,
            "bert_label" :bert_label,
            "segment_label": segment_label,
            "is_next": is_next_label
        }
        return {key:torch.tensor(value) for key,value in output.items()}
    
    def rendom_word(self,sentence):
        tokens=sentence.split()
        output_label=[]
        output=[]

        for i,token in enumerate(tokens):
            prob=random.random()

            token_id =self.tokenizer(token)['input_ids'][1:-1]

            if porb <0.15:
                porb /=0.15

                if porb <0.8:
                    for i in range(len(token_id)):
                        output.append(self.tokenizer.vocab['[MASK]'])

                elif porb <0.9:
                    for i in range(len(token_id)):
                        output.append(random.randrange(len(self.tokenizer.vocab)))

                else:
                    output.append(token_id)
                    for i in range(len(token_id)):
                        output_label.append(0)
            

        output=list(itertools.chain(*[[x] if not isinstance(x,list) else x for x in output]))
        output_label=list(itertools.chain(*[[x] if not isinstance(x,list) else x for x in output_label]))
        assert len(output) ==len(output_label)
        return output,output_label
    
    def get_sent(self,index):
        t1,t2=self.get_corpus_line(index)

        if random.random() > 0.5:
            return t1,t2,1
        
        else:
            return t1,self.get_random_line(),0
        
    def get_corpus_line(self,item):
        return self.lines[item][0],self.lines[item][1]
    
    def get_random_line(self):
        return self.lines[random.randrange(len(self.lines))][1]

In [ ]:
class PositionalEmbedding(torch.nn.model):
    def _init__(self,d_model,max_len=128):
        super().__init__()

        pe=torch.zeros(max_len,d_model).float()
        pe.require_grad=False

        for pos in range(max_len):
            for i in range(0,d_model,2):
                pe[pos,i]=math.sin(pos/(10000 **((2*i)/d_model)))
                pe[pos,i+1]= math.cos(pos/(10000 **((2*(i+1)))/d_model))

            self.pe=pe.unsqueeze(0)

        def forward(self,x):
            return self.pe
        
    class BERTEmbedding(torch.nn.Model):
        def __init__(self,vocab_size,embed_size,seq_len=64,dropout=0.1):
            super().__init__()
            self.embed_size=embed_size
            self.token= torch.nn.Embedding(vocab_size,embed_size,padding_idx=0)
            self.segemnt = torch.nn.Embedding(3,embed_size,padding_idx=0)
            self.position=PositionalEmbedding(d_model=embed_size,max_len=seq_len)
            self.dropout=torch.nn.Dropout(p=dropout)

            def forward(self,sequence,segment_label):
                x=self.token(sequence)+self.postion(sequence)+self.segment(segment_label)
                return self.dropout(x)
            

## BERT Architecture:
BERT needs on only encoder of transformaer as it's goal is to understand contextual reletionships between words in a text.


In [ ]:
class MultiHeadedAttention(torch.nn.Module):
    def __init__(self,heads,d_model,dropout=0.1):
        super(MultiHeadedAttention,self).__init__()

        assert d_model% heads ==0
        self.d_k=d_model//heads
        self.heads=heads
        self.dropout=torch.nn.Dropout(dropout)

        self.query= torch.nn.Linear(d_model,d_model)
        self.key = torch.nn.Linear(d_model,d_model)
        self.value=torch.nn.Linear(d_model,d_model)
        self.output_linear=torch.nn.Linear(d_model,d_model)

    def forward(self,query,key,value,mask):
        query= self.query(query)
        key=self.key(key)
        value=self.value(value)

        query=query.view(query.shape[0],-1,self.heads,self.d_k).permute(0,2,1,3)
        key=key.view(key.shape[0],-1,self.heads,self.d_k).permute(0,2,1,3)
        value=value.view(value.shape[0],-1,self.heads,self.d_k).permute(0,2,1,3)

        scores = torch.matmul(query,key.permute(0,1,3,2))/math.sqrt(query.size(-1))

        scores= scores.masked_fill(mask==0,-1e9)

        weights =F.softmax(scores,dim=1)
        weights=self.dropout(weights)

        context=torch.matmul(weights,value)

        context = context.permute(0,2,1,3).contiguous().view(context.shape[0],-1,self.heads*self.d_k)

        return self.output_liner(context)
    

class FeedForward(torch.nn.Module):

    def __init__(self,d_module,middle_dim=2048,dropout=0.1):
        super(FeedForward,self).__init__()

        self.fc1 =torch.nn.Linear(d_module,d_module)

        self.fc2 = torch.nn.Linear(middle_dim,d_module)

        self.dropout =torch.nn.Dropout(dropout)
        self.activation=torch.nn.GELU()

    def forward(self, embeddings, mask):
        # embeddings: (batch_size, max_len, d_model)
        # encoder mask: (batch_size, 1, 1, max_len)
        # result: (batch_size, max_len, d_model)
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        # residual layer
        interacted = self.layernorm(interacted + embeddings)
        # bottleneck
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded

In [ ]:
class BERT(torch.nn.Module):
    """
    BERT model : Bidirectional Encoder Representations from Transformers.
    """

    def __init__(self, vocab_size, d_model=768, n_layers=12, heads=12, dropout=0.1):
        """
        :param vocab_size: vocab_size of total words
        :param hidden: BERT model hidden size
        :param n_layers: numbers of Transformer blocks(layers)
        :param attn_heads: number of attention heads
        :param dropout: dropout rate
        """

        super().__init__()
        self.d_model = d_model
        self.n_layers = n_layers
        self.heads = heads

        # paper noted they used 4 * hidden_size for ff_network_hidden_size
        self.feed_forward_hidden = d_model * 4

        # embedding for BERT, sum of positional, segment, token embeddings
        self.embedding = BERTEmbedding(vocab_size=vocab_size, embed_size=d_model)

        # multi-layers transformer blocks, deep network
        self.encoder_blocks = torch.nn.ModuleList(
            [EncoderLayer(d_model, heads, d_model * 4, dropout) for _ in range(n_layers)])

    def forward(self, x, segment_info):
        # attention masking for padded token
        # (batch_size, 1, seq_len, seq_len)
        mask = (x > 0).unsqueeze(1).repeat(1, x.size(1), 1).unsqueeze(1)

        # embedding the indexed sequence to sequence of vectors
        x = self.embedding(x, segment_info)

        # running over multiple transformer blocks
        for encoder in self.encoder_blocks:
            x = encoder.forward(x, mask)
        return x

class NextSentencePrediction(torch.nn.Module):
    """
    2-class classification model : is_next, is_not_next
    """

    def __init__(self, hidden):
        """
        :param hidden: BERT model output size
        """
        super().__init__()
        self.linear = torch.nn.Linear(hidden, 2)
        self.softmax = torch.nn.LogSoftmax(dim=-1)

    def forward(self, x):
        # use only the first token which is the [CLS]
        return self.softmax(self.linear(x[:, 0]))

class MaskedLanguageModel(torch.nn.Module):
    """
    predicting origin token from masked input sequence
    n-class classification problem, n-class = vocab_size
    """

    def __init__(self, hidden, vocab_size):
        """
        :param hidden: output size of BERT model
        :param vocab_size: total vocab size
        """
        super().__init__()
        self.linear = torch.nn.Linear(hidden, vocab_size)
        self.softmax = torch.nn.LogSoftmax(dim=-1)

    def forward(self, x):
        return self.softmax(self.linear(x))

class BERTLM(torch.nn.Module):
    """
    BERT Language Model
    Next Sentence Prediction Model + Masked Language Model
    """

    def __init__(self, bert: BERT, vocab_size):
        """
        :param bert: BERT model which should be trained
        :param vocab_size: total vocab size for masked_lm
        """

        super().__init__()
        self.bert = bert
        self.next_sentence = NextSentencePrediction(self.bert.d_model)
        self.mask_lm = MaskedLanguageModel(self.bert.d_model, vocab_size)

    def forward(self, x, segment_label):
        x = self.bert(x, segment_label)
        return self.next_sentence(x), self.mask_lm(x)
